In [4]:
import pandas as pd
import numpy as np
import json

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [9]:
LC_GRID = '//div[@class="grid grid-cols-4 sm:grid-cols-5 md:grid-cols-6 lg:grid-cols-7 xl:grid-cols-8 h-100 p-3 text-slate-950"]'

class LCStatScraper:
    def __init__(self, url):
        self.driver = None
        self.url = url

    def create_driver(self):
        self.driver = webdriver.Chrome()
        self.wait = WebDriverWait(self.driver, 10)
        self.driver.get(self.url)
        self.driver.maximize_window()
    
    def mine_lcs(self):
        lc_grid = self.wait.until(EC.presence_of_element_located((By.XPATH, LC_GRID)))
        lc_list = lc_grid.find_elements(By.TAG_NAME, 'a')
        return_dict = {}
        for i in range(len(lc_list)):
            lc_dict = {}

            # refresh references
            lc_grid = self.wait.until(EC.presence_of_element_located((By.XPATH, LC_GRID)))
            lc_list = lc_grid.find_elements(By.TAG_NAME, 'a')

            # get name and ID
            lc_link = lc_list[i]
            
            name = lc_link.find_element(By.TAG_NAME, 'div').get_attribute('innerText')
            id = lc_link.get_attribute('href').split('/')[-1]

            image_link = lc_link.find_element(By.TAG_NAME, 'img').get_attribute('src')

            lc_dict['Name'] = name
            lc_dict['ImageLink'] = image_link
            return_dict[id] = lc_dict
        self.driver.close()
        return return_dict

In [11]:
HAKUSHIN_URL = 'https://hsr2.hakush.in/lightcone'

scraper = LCStatScraper(HAKUSHIN_URL)
scraper.create_driver()
lc_dict = scraper.mine_lcs()

In [18]:
with open('../datamine/EquipmentPromotionConfig.json') as f:
    json_data = json.load(f)

def organize_lc_data(data, id):
    ascensions = [
        data[id]['0'],
        data[id]['1'],
        data[id]['2'],
        data[id]['3'],
        data[id]['4'],
        data[id]['5'],
        data[id]['6']
    ]
    
    level_data = {}

    # handle this in sections
    for i in range(len(ascensions)):
        current_ascension = ascensions[i]
        min_level, max_level = 1, 0
        if i == 0:
            max_level = ascensions[i]['MaxLevel']
        else:
            min_level = ascensions[i-1]['MaxLevel']
            max_level = ascensions[i]['MaxLevel']

        base_atk = current_ascension['BaseAttack']['Value']
        base_def = current_ascension['BaseDefence']['Value']
        base_hp = current_ascension['BaseHP']['Value']

        atk_add = current_ascension['BaseAttackAdd']['Value']
        def_add = current_ascension['BaseDefenceAdd']['Value']
        hp_add = current_ascension['BaseHPAdd']['Value']
        
        # now that we know min and max level, fill in the values
        for j in range(min_level, max_level+1):
            stat_object = {}
            current_atk = round((base_atk + atk_add*(j-1)), 3)
            current_def = round((base_def + def_add*(j-1)), 3)
            current_hp = round((base_hp + hp_add*(j-1)), 3)
            level_tag = str(j)
            if (j == min_level) & (j != 1):
                # we're doing the x+
                level_tag = str(min_level) + '+'
            stat_object['ATK'] = current_atk
            stat_object['DEF'] = current_def
            stat_object['HP'] = current_hp
            level_data[level_tag] = stat_object
    return level_data

organize_lc_data(json_data, '20000')

KeyError: 'AttackBase'